L2-regularized logistic regression for binary or multiclass classification; trains a model (on `train.txt`), optimizes L2 regularization strength on `dev.txt`, and evaluates performance on `test.txt`.  Reports test accuracy with 95% confidence intervals and prints out the strongest coefficients for each class.

In [1]:
from scipy import sparse
from sklearn import linear_model
from collections import Counter
import numpy as np
import operator
import nltk
import math
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler

In [2]:
!python3 -m nltk.downloader punkt

/Users/azhara001/opt/anaconda3/envs/deeplearning_env/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /Users/azhara001/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def load_data(filename):
    X = []
    Y = []
    with open(filename, encoding="utf-8") as file:
        for line in file:
            cols = line.split("\t")
            idd = cols[0]
            label = cols[2].lstrip().rstrip()
            text = cols[3]

            X.append(text)
            Y.append(label)

    return X, Y


In [4]:
class Classifier:

    def __init__(self, feature_method, feature_kwargs, trainX, trainY, devX, devY, testX, testY):
        self.feature_vocab = {}
        self.feature_method = feature_method
        self.feature_kwargs = feature_kwargs
        self.min_feature_count=2
        self.log_reg = None

        self.trainY=trainY
        self.devY=devY
        self.testY=testY
        
        self.trainX = self.process(trainX, training=True)
        self.devX = self.process(devX, training=False)
        self.testX = self.process(testX, training=False)

    # Featurize entire dataset
    def featurize(self, data):
        featurized_data = []
        for text in data:
            feats = self.feature_method(text, **self.feature_kwargs)
            featurized_data.append(feats)
        return featurized_data

    # Read dataset and returned featurized representation as sparse matrix + label array
    def process(self, X_data, training = False):
        
        data = self.featurize(X_data)

        if training:
            fid = 0
            feature_doc_count = Counter()
            for feats in data:
                for feat in feats:
                    feature_doc_count[feat]+= 1

            for feat in feature_doc_count:
                if feature_doc_count[feat] >= self.min_feature_count:
                    self.feature_vocab[feat] = fid
                    fid += 1

        F = len(self.feature_vocab)
        D = len(data)
        X = sparse.dok_matrix((D, F))
        for idx, feats in enumerate(data):
            for feat in feats:
                if feat in self.feature_vocab:
                    X[idx, self.feature_vocab[feat]] = feats[feat]

        return X


    # Train model and evaluate on held-out data
    def train(self):
        scaler = StandardScaler(with_mean=False)  # Use with_mean=False for sparse matrix compatibility
        self.trainX = scaler.fit_transform(self.trainX)
        self.devX = scaler.transform(self.devX)
        self.testX = scaler.transform(self.testX)
        (D,F) = self.trainX.shape
        best_dev_accuracy=0
        best_model=None
        for C in [0.1, 1, 10, 100]:
            self.log_reg = linear_model.LogisticRegression(C = C, max_iter=1000)
            self.log_reg.fit(self.trainX, self.trainY)
            training_accuracy = self.log_reg.score(self.trainX, self.trainY)
            development_accuracy = self.log_reg.score(self.devX, self.devY)
            if development_accuracy > best_dev_accuracy:
                best_dev_accuracy=development_accuracy
                best_model=self.log_reg

#             print("C: %s, Train accuracy: %.3f, Dev accuracy: %.3f" % (C, training_accuracy, development_accuracy))

        self.log_reg=best_model
        

    def test(self):
        return self.log_reg.score(self.testX, self.testY)
        

    def printWeights(self, n=10):

        reverse_vocab=[None]*len(self.log_reg.coef_[0])
        for k in self.feature_vocab:
            reverse_vocab[self.feature_vocab[k]]=k

        # binary
        if len(self.log_reg.classes_) == 2:
              weights=self.log_reg.coef_[0]

              cat=self.log_reg.classes_[1]
              for feature, weight in list(reversed(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

              cat=self.log_reg.classes_[0]
              for feature, weight in list(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1)))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

        # multiclass
        else:
          for i, cat in enumerate(self.log_reg.classes_):

              weights=self.log_reg.coef_[i]

              for feature, weight in list(reversed(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

            

In [5]:
import numpy as np
from scipy import sparse
from nltk.tokenize import word_tokenize
import spacy
import gensim.downloader as api
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load resources
nlp = spacy.load("en_core_web_sm")  # For NER
word_vectors = api.load("word2vec-google-news-300")  # For Word Embeddings
analyzer = SentimentIntensityAnalyzer()  # For Sentiment Analysis


2024-04-28 13:37:12.506604: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [6]:

def featurize(text, **kwargs):
    feats = {}
    words = word_tokenize(text.lower())

#     # Basic BOW features
    if kwargs.get('bag_of_words', True):
        for word in words:
            feats[f"word_{word}"] = 1

    # Sentiment features
    if kwargs.get('sentiment', False):
        sentiment = analyzer.polarity_scores(text)
        feats.update({
            'sentiment_neg': sentiment['neg'],
            'sentiment_neu': sentiment['neu'],
            'sentiment_pos': sentiment['pos'],
            'sentiment_compound': sentiment['compound']
        })

    # Word embeddings (averaged)
    if kwargs.get('word_embed', False):
        embeddings = [word_vectors[word] for word in words if word in word_vectors]
        if embeddings:
            avg_embedding = np.mean(embeddings, axis=0)
            for i, value in enumerate(avg_embedding):
                feats[f"embedding_{i}"] = value

    # NER features
    if kwargs.get('ner', False):
        doc = nlp(text)
        for ent in doc.ents:
            feats[f"ner_{ent.label_}_{ent.text}"] = 1
            
    if kwargs.get('hyperbolae', False):
        hyperbolic_count = count_hyperbolic_terms(text)
        feats['hyperbolic_count'] = hyperbolic_count
        
    if kwargs.get('topics', False):
        topics = get_document_topics(text)
        for topic_id, weight in topics:
            if weight > 0.5:
                feats[f"topic_{topic_id}"] = weight
            
    return feats



In [7]:
def confidence_intervals(accuracy, n, significance_level):
    critical_value=(1-significance_level)/2
    z_alpha=-1*norm.ppf(critical_value)
    se=math.sqrt((accuracy*(1-accuracy))/n)
    return accuracy-(se*z_alpha), accuracy+(se*z_alpha)

In [8]:
def run(trainingFile, devFile, testFile, featurize_kwargs={}):
    trainX, trainY=load_data(trainingFile)
    devX, devY=load_data(devFile)
    testX, testY=load_data(testFile)
    
    simple_classifier = Classifier(featurize, featurize_kwargs, trainX, trainY, devX, devY, testX, testY)
    simple_classifier.train()
    accuracy=simple_classifier.test()
    
    lower, upper=confidence_intervals(accuracy, len(testY), .95)
    print("Test accuracy for best dev model: %.3f, 95%% CIs: [%.3f %.3f]\n" % (accuracy, lower, upper))

    simple_classifier.printWeights()
    return accuracy
    


# Make train test splits

In [9]:
import pandas as pd
adj_data = pd.read_table("./AP2/adjudicated_data.txt")
adj_data

,1,adjudicated,humor,woman who teaches special-needs children killing it at dinner party
0,2,adjudicated,satire,dnc criticized for overly restrictive debate r...
1,3,adjudicated,irony,custodian taken into custody
2,4,adjudicated,none,mike pence takes oath of office as country's n...
3,5,adjudicated,humor,bedtime story from fucking bible again
4,6,adjudicated,satire,literary theorists admit they still have no id...
...,...,...,...,...
494,496,adjudicated,none,longing to finally visit the cuba of my dreams
495,497,adjudicated,none,trump's evangelical advisors urged him to prot...
496,498,adjudicated,none,flaws in how we evaluate leaders (from kahnema...
497,499,adjudicated,irony,intel unveils oversized novelty processor


In [10]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(adj_data, test_size=100)
train, dev = train_test_split(train, test_size=100)
len(train), len(dev), len(test)

(299, 100, 100)

In [11]:
train.to_csv("./splits/train.txt", sep="\t", index=False)
dev.to_csv("./splits/dev.txt", sep="\t", index=False)
test.to_csv("./splits/test.txt", sep="\t", index=False)

In [12]:
trainingFile = "splits/train.txt"
devFile = "splits/dev.txt"
testFile = "splits/test.txt"

featurize_kwargs = {
    'bag_of_words': True
}
    
run(trainingFile, devFile, testFile, featurize_kwargs)


Test accuracy for best dev model: 0.495, 95% CIs: [0.398 0.593]

humor	0.316	word_man
humor	0.252	word_study
humor	0.237	word_toddler
humor	0.196	word_nothing
humor	0.188	word_argument
humor	0.180	word_of
humor	0.179	word_inside
humor	0.170	word_teacher
humor	0.165	word_area
humor	0.163	word_fighting

irony	0.254	word_introduces
irony	0.226	word_health
irony	0.198	word_unveils
irony	0.190	word_thinking
irony	0.190	word_favorite
irony	0.190	word_life
irony	0.166	word_world
irony	0.156	word_floor
irony	0.153	word_older
irony	0.139	word_another

none	0.205	word_.
none	0.203	word_college
none	0.201	word_a
none	0.201	word_in
none	0.199	word_aide
none	0.196	word_kids
none	0.196	word_why
none	0.183	word_the
none	0.179	word_even
none	0.179	word_'s

satire	0.280	word_for
satire	0.202	word_they
satire	0.199	word_at
satire	0.192	word_women
satire	0.190	word_doctors
satire	0.187	word_'ll
satire	0.178	word_features
satire	0.177	word_just
satire	0.174	word_only
satire	0.162	word_saliva



0.49504950495049505

## Improving on the baseline

## Sentiment

In [13]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def sentiment_scores(text):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(text)
    return scores



## Word Embeddings

In [14]:
import gensim.downloader as api
from nltk.tokenize import word_tokenize

# Load pre-trained Word2Vec model.
word_vectors = api.load("word2vec-google-news-300")

def get_word_embedding(text):
    words = word_tokenize(text.lower())
    embeddings = [word_vectors[word] for word in words if word in word_vectors]
    # Average the word vectors of all words in the text
    if embeddings:
        return sum(embeddings) / len(embeddings)
    else:
        return np.zeros(300)  # Return zero vector if no words have embeddings


## NER

In [15]:
import spacy

# Load an English NLP model
nlp = spacy.load("en_core_web_sm")

def named_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]


## Topic Features

In [16]:
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
nltk.download('punkt')

trainingFile = "splits/train.txt"
devFile = "splits/dev.txt"

def preprocess_text(text):
    # Lowercase, remove special characters, and tokenize text
    tokens = word_tokenize(re.sub(r'[\W_]+', ' ', text.lower()))
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and len(token) > 1]
    return tokens

def read_documents(file_path):
    documents = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            cells = line.split("\t")
            documents.append(preprocess_text(cells[-1]))
            
    return documents

# Read and preprocess the training and dev data
train_documents = read_documents(trainingFile)
dev_documents = read_documents(devFile)

# Combine train and dev documents if you want to use both for training
documents = train_documents + dev_documents

# Assuming 'documents' is a list of tokenized documents (list of list of tokens)
dictionary = Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]

# Train LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, random_state=42)


def get_document_topics(text):
    tokens = preprocess_text(text)
    # Get the bag-of-words representation for the text
    bow = dictionary.doc2bow(tokens)
    # Retrieve the list of topics with their probabilities
    topics = lda_model.get_document_topics(bow)

    return topics


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/azhara001/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/azhara001/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Hyperbolic Terms

In [17]:
hyperbolic_terms = ["unbelievable", "amazing", "incredible", "never", "always", "worst", "best"]

def count_hyperbolic_terms(text):
    words = word_tokenize(text.lower())
    count = sum(1 for word in words if word in hyperbolic_terms)
    return count


In [18]:
from itertools import product
from tqdm import tqdm

params = {
    'bag_of_words': [True, False],
    'sentiment': [True, False],
    'word_embed': [True, False],
    'ner': [True, False],
    'hyperbolae': [True, False],
    'topics': [True, False]
}

keys = params.keys()
values = (params[key] for key in keys)
kwargs_combinations = [dict(zip(keys, combination)) 
                for combination in product(*values) 
                if any(combination)]
# featurize_kwargs = {
#     'bag_of_words': False,
#     'sentiment': False,
#     'word_embed': True,
#     'ner': True, 
#     'hyperbolae': False,
#     'topics': True
# }
    
# run(trainingFile, devFile, testFile, featurize_kwargs)

results = []

for kwarg_combination in tqdm(kwargs_combinations):
    # Run the model with the current combination of features
    accuracy = run(trainingFile, devFile, testFile,kwarg_combination)
    results.append((kwarg_combination, accuracy))

# Find the best performing configuration
best_combination = max(results, key=lambda x: x[1])

print("Best configuration:", best_combination[0])
print("Highest accuracy:", best_combination[1])



  2%|▏         | 1/63 [00:05<06:00,  5.81s/it]

Test accuracy for best dev model: 0.495, 95% CIs: [0.398 0.593]

humor	0.250	word_nothing
humor	0.219	word_argument
humor	0.218	word_running
humor	0.217	word_negative
humor	0.214	word_toddler
humor	0.214	word_wild
humor	0.209	word_democratic
humor	0.206	word_area
humor	0.194	word_teacher
humor	0.190	word_if

irony	0.290	word_floor
irony	0.261	word_another
irony	0.258	word_thinking
irony	0.241	word_so
irony	0.230	word_world
irony	0.212	word_introduces
irony	0.207	word_health
irony	0.205	word_other
irony	0.189	embedding_58
irony	0.189	embedding_24

none	0.315	word_the
none	0.304	word_eye
none	0.289	embedding_156
none	0.285	word_.
none	0.263	embedding_275
none	0.263	embedding_188
none	0.262	embedding_25
none	0.261	embedding_101
none	0.259	word_even
none	0.257	embedding_13

satire	0.337	word_features
satire	0.335	embedding_74
satire	0.301	word_for
satire	0.287	word_doctors
satire	0.273	embedding_293
satire	0.266	word_just
satire	0.255	word_possible
satire	0.244	word_'ll
satire	0.240	embedd

  3%|▎         | 2/63 [00:11<05:35,  5.50s/it]

Test accuracy for best dev model: 0.535, 95% CIs: [0.437 0.632]

humor	0.181	word_nothing
humor	0.163	word_argument
humor	0.162	word_wild
humor	0.162	word_area
humor	0.161	word_negative
humor	0.161	word_running
humor	0.160	word_toddler
humor	0.159	word_democratic
humor	0.143	word_teacher
humor	0.140	embedding_27

irony	0.223	word_floor
irony	0.195	word_thinking
irony	0.186	word_another
irony	0.176	word_so
irony	0.165	word_world
irony	0.164	word_introduces
irony	0.152	word_other
irony	0.148	word_health
irony	0.147	embedding_58
irony	0.134	embedding_24

none	0.226	word_the
none	0.223	word_eye
none	0.212	word_.
none	0.210	embedding_156
none	0.197	embedding_13
none	0.197	embedding_101
none	0.194	embedding_188
none	0.193	word_even
none	0.190	embedding_25
none	0.190	embedding_275

satire	0.239	embedding_74
satire	0.234	word_features
satire	0.220	word_for
satire	0.217	word_doctors
satire	0.208	embedding_293
satire	0.191	word_just
satire	0.178	word_'ll
satire	0.177	embedding_154
satire	0.177	e

  5%|▍         | 3/63 [00:16<05:19,  5.32s/it]

...

Test accuracy for best dev model: 0.396, 95% CIs: [0.301 0.491]

humor	0.182	ner_NORP_americans
humor	0.126	ner_PERSON_god
humor	0.119	ner_CARDINAL_4
humor	0.073	ner_ORDINAL_third
humor	-0.020	ner_PERSON_bob corker
humor	-0.053	ner_PERSON_taylor
humor	-0.053	ner_GPE_new york
humor	-0.053	ner_PERSON_mike
humor	-0.053	ner_GPE_france
humor	-0.053	ner_GPE_west virginia

irony	0.156	ner_CARDINAL_3
irony	0.153	ner_PERSON_donald trump
irony	0.119	ner_PERSON_clinton
irony	0.113	ner_ORDINAL_third
irony	0.055	ner_GPE_u.s.
irony	-0.018	ner_PERSON_bob corker
irony	-0.030	ner_CARDINAL_2
irony	-0.040	ner_PERSON_taylor
irony	-0.040	ner_GPE_new york
irony	-0.040	ner_PERSON_mike

none	0.245	ner_ORDINAL_first
none	0.212	ner_CARDINAL_two
none	0.182	ner_CARDINAL_2
none	0.173	ner_PERSON_taylor
none	0.173	ner_GPE_new york
none	0.173	ner_PERSON_mike
none	0.173	ner_GPE_france
none	0.173	ner_GPE_west virginia
none	0.160	ner_ORG_gop
none	0.131	ner_PERSON_bob corker

satire	0.082	ner_GPE_iran
satire	0.079	ner_CA

 98%|█████████▊| 62/63 [03:04<00:01,  1.50s/it]

Test accuracy for best dev model: 0.406, 95% CIs: [0.310 0.502]

humor	0.145	hyperbolic_count
humor	-0.090	topic_7
humor	-0.094	topic_2
humor	-0.107	topic_6
humor	-0.172	topic_5
humor	-0.221	topic_0
humor	-0.234	topic_1
humor	-0.336	topic_9
humor	-0.362	topic_4
humor	-0.398	topic_8

irony	-0.201	topic_7
irony	-0.206	topic_3
irony	-0.254	topic_8
irony	-0.305	topic_6
irony	-0.357	topic_5
irony	-0.435	topic_4
irony	-0.454	topic_0
irony	-0.525	topic_9
irony	-0.541	topic_1
irony	-0.548	topic_2

none	0.302	hyperbolic_count
none	-0.001	topic_3
none	-0.289	topic_4
none	-0.453	topic_9
none	-0.474	topic_1
none	-0.498	topic_6
none	-0.513	topic_8
none	-0.590	topic_0
none	-0.598	topic_2
none	-0.654	topic_5

satire	1.313	topic_9
satire	1.294	topic_3
satire	1.265	topic_0
satire	1.250	topic_1
satire	1.241	topic_2
satire	1.184	topic_5
satire	1.166	topic_8
satire	1.086	topic_4
satire	0.950	topic_7
satire	0.910	topic_6

Test accuracy for best dev model: 0.406, 95% CIs: [0.310 0.502]

humor	0.009	hyperbol

100%|██████████| 63/63 [03:04<00:00,  2.93s/it]

Test accuracy for best dev model: 0.406, 95% CIs: [0.310 0.502]

humor	-0.092	topic_7
humor	-0.096	topic_2
humor	-0.110	topic_6
humor	-0.175	topic_5
humor	-0.224	topic_0
humor	-0.237	topic_1
humor	-0.338	topic_9
humor	-0.363	topic_4
humor	-0.401	topic_8
humor	-1.092	topic_3

irony	-0.182	topic_7
irony	-0.187	topic_3
irony	-0.242	topic_8
irony	-0.304	topic_6
irony	-0.345	topic_5
irony	-0.418	topic_4
irony	-0.453	topic_0
irony	-0.510	topic_9
irony	-0.528	topic_2
irony	-0.528	topic_1

none	-0.022	topic_3
none	-0.311	topic_4
none	-0.473	topic_9
none	-0.492	topic_1
none	-0.501	topic_6
none	-0.529	topic_8
none	-0.594	topic_0
none	-0.624	topic_2
none	-0.671	topic_5
none	-0.683	topic_7

satire	1.320	topic_9
satire	1.301	topic_3
satire	1.271	topic_0
satire	1.257	topic_1
satire	1.248	topic_2
satire	1.191	topic_5
satire	1.172	topic_8
satire	1.092	topic_4
satire	0.957	topic_7
satire	0.915	topic_6

Best configuration: {'bag_of_words': True, 'sentiment': True, 'word_embed': True, 'ner': True, 'hyper

In [19]:
results

[({'bag_of_words': True,
   'sentiment': True,
   'word_embed': True,
   'ner': True,
   'hyperbolae': True,
   'topics': True},
  0.49504950495049505),
 ({'bag_of_words': True,
   'sentiment': True,
   'word_embed': True,
   'ner': True,
   'hyperbolae': True,
   'topics': False},
  0.5346534653465347),
 ({'bag_of_words': True,
   'sentiment': True,
   'word_embed': True,
   'ner': True,
   'hyperbolae': False,
   'topics': True},
  0.5148514851485149),
 ({'bag_of_words': True,
   'sentiment': True,
   'word_embed': True,
   'ner': True,
   'hyperbolae': False,
   'topics': False},
  0.5346534653465347),
 ({'bag_of_words': True,
   'sentiment': True,
   'word_embed': True,
   'ner': False,
   'hyperbolae': True,
   'topics': True},
  0.49504950495049505),
 ({'bag_of_words': True,
   'sentiment': True,
   'word_embed': True,
   'ner': False,
   'hyperbolae': True,
   'topics': False},
  0.5247524752475248),
 ({'bag_of_words': True,
   'sentiment': True,
   'word_embed': True,
   'ner':